# Analyse des données de transport

## Objectifs

Ce notebool a pour objectif d'explorer et d'analyser un historique de tickets de transports:
* Analyser les prix et durée des trajets
* comparer les modes de transport selon la distance
* identifier des tendances et incohérences dans les données
* proposer des analyses et visualisations pertinentes

## Outils et bibliothèques utilisés
* Python
* pandas
* numpy
* matplotlib
* scikit-learn
* pyarrow
* pyjanitor
* ipykernel
* jupyter
* seaborn
* polars
* plotly
* folium
* missingno
* great_expectations

## Auteur
Haja Rabemananjara

## Date
17 janvier 2026

## 1. Import des librairies et configuration de l'apparence globale des graphiques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
import statsmodel

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

## 2. Chargement des données

In [82]:
ticket = pd.read_csv('data/ticket_data.csv')
cities = pd.read_csv('data/cities.csv')
stations = pd.read_csv('data/stations.csv')
providers = pd.read_csv('data/providers.csv')

###### Aperçu des données

In [83]:
print("Dimensions des datasets:")
{ticket.shape, cities.shape, stations.shape, providers.shape}

Dimensions des datasets:


{(227, 10), (8040, 6), (11035, 4), (74168, 12)}

In [84]:
from IPython.display import display, Markdown

display(Markdown("### Info sur les tickets"))
ticket.head()
ticket.info()

display(Markdown("### Info sur les villes"))
cities.head()
cities.info()

display(Markdown("### Info sur les stations"))
stations.head()
stations.info()

display(Markdown("### Info sur les fournisseurs"))
providers.head()
providers.info()

### Info sur les tickets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74168 entries, 0 to 74167
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               74168 non-null  int64  
 1   company          74168 non-null  int64  
 2   o_station        32727 non-null  float64
 3   d_station        32727 non-null  float64
 4   departure_ts     74168 non-null  object 
 5   arrival_ts       74168 non-null  object 
 6   price_in_cents   74168 non-null  int64  
 7   search_ts        74168 non-null  object 
 8   middle_stations  32727 non-null  object 
 9   other_companies  32727 non-null  object 
 10  o_city           74168 non-null  int64  
 11  d_city           74168 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 6.8+ MB


### Info sur les villes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8040 entries, 0 to 8039
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8040 non-null   int64  
 1   local_name   8040 non-null   object 
 2   unique_name  8039 non-null   object 
 3   latitude     8040 non-null   float64
 4   longitude    8040 non-null   float64
 5   population   369 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 377.0+ KB


### Info sur les stations

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11035 entries, 0 to 11034
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           11035 non-null  int64  
 1   unique_name  11035 non-null  object 
 2   latitude     11035 non-null  float64
 3   longitude    11035 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 345.0+ KB


### Info sur les fournisseurs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    227 non-null    int64 
 1   company_id            227 non-null    int64 
 2   provider_id           213 non-null    object
 3   name                  227 non-null    object
 4   fullname              227 non-null    object
 5   has_wifi              224 non-null    object
 6   has_plug              224 non-null    object
 7   has_adjustable_seats  224 non-null    object
 8   has_bicycle           224 non-null    object
 9   transport_type        227 non-null    object
dtypes: int64(2), object(8)
memory usage: 17.9+ KB


In [85]:
print("Description statistique des tickets")
ticket.describe()

Description statistique des tickets


,id,company,o_station,d_station,price_in_cents,o_city,d_city
count,7.42e+04,74168.00,32727.00,32727.00,74168.00,74168.00,74168.00
mean,6.83e+06,7109.57,2907.13,2347.86,4382.71,849.19,883.78
std,2.14e+04,3005.38,3347.63,3090.80,3739.33,1485.79,1654.70
min,6.80e+06,9.00,3.00,3.00,300.00,5.00,1.00
25%,6.81e+06,8376.00,400.00,396.00,1900.00,485.00,453.00
50%,6.83e+06,8385.00,701.00,575.00,3350.00,628.00,562.00
75%,6.85e+06,8385.00,6246.00,4538.00,5250.00,628.00,628.00
max,6.87e+06,8387.00,11017.00,11017.00,38550.00,12190.00,12190.00


## 3. Compréhension métier des données

### Description des colonnes clés
* *price_in_cents* : prix du ticket en cents (€)
* *departure_ts / arrival_ts* : départ et arrivée : format data YYYY-mm-jj HH:mm:ss
* *o_city / d_city* : ville de départ / arrivée
* *o_station / d_station* : station de départ / arrivée
* *company* : provider (ex : TGV, TER, Blablacar)

## 4. Nettoyage de données

#### Valeurs manquantes

In [86]:
print('Vérification des valeurs manquantes dans les tickets')
ticket.isna().sum()

Vérification des valeurs manquantes dans les tickets


id                     0
company                0
o_station          41441
d_station          41441
departure_ts           0
arrival_ts             0
price_in_cents         0
search_ts              0
middle_stations    41441
other_companies    41441
o_city                 0
d_city                 0
dtype: int64

On constate que 44 % (41441 / 74168) n'ont pas d'information sur les stations ('o_station' , 'd_station').
Cela correspond probablement aux trajets en covoiturage qui ne passent pas par des stations fixes.

In [87]:
missing_data = pd.DataFrame({
    'Dataset': ['Ticket', 'Cities', 'Stations', 'Providers'],
    'Total Rows': [len(ticket), len(cities), len(stations), len(providers)],
    'Missing Values': [ticket.isnull().sum().sum(), cities.isnull().sum().sum(), stations.isnull().sum().sum(), providers.isnull().sum().sum()]
})
print("Résumé des valeurs manquantes")
missing_data

Résumé des valeurs manquantes


,Dataset,Total Rows,Missing Values
0,Ticket,74168,165764
1,Cities,8040,7672
2,Stations,11035,0
3,Providers,227,26


Détail des valeurs manquantes dans le dataset des tickets:"

In [88]:
missing_ticket = ticket.isnull().sum()
missing_ticket_pct = ((missing_ticket / len(ticket)) * 100).round(2)
missing_ticket_df = pd.DataFrame({
    'Valeurs manquantes': missing_ticket,
    'Pourcentage (%)': missing_ticket_pct
})
missing_ticket_df = missing_ticket_df[missing_ticket_df['Valeurs manquantes'] > 0]
missing_ticket_df.sort_values('Valeurs manquantes', ascending=False)

,Valeurs manquantes,Pourcentage (%)
o_station,41441,55.87
d_station,41441,55.87
middle_stations,41441,55.87
other_companies,41441,55.87


#### Doublons

La suppression des doublons permettra d'éviter que les résultats soient biaisés.

In [89]:
ticket = ticket.drop_duplicates()

if ticket.drop_duplicates().shape[0] == ticket.shape[0]:
    print("Aucun doublon trouvé.")
else:
    print('Après suppression des doublons :', ticket.shape)

Aucun doublon trouvé.


#### Conversion

#### Conversion des prix en euros

In [90]:
ticket['price_eur'] = ticket['price_in_cents'] / 100
ticket[['price_in_cents', 'price_eur']].head()

,price_in_cents,price_eur
0,4550,45.5
1,1450,14.5
2,7400,74.0
3,13500,135.0
4,7710,77.1


#### Conversion des timestamps

In [91]:
ticket['departure_ts'] = pd.to_datetime(ticket['departure_ts'], format='mixed')
ticket['arrival_ts'] = pd.to_datetime(ticket['arrival_ts'], format='mixed')
ticket['search_ts'] = pd.to_datetime(ticket['search_ts'], format='mixed')

Calcul de la durée des trajets en minutes

In [92]:
ticket['duration_min'] = (ticket['arrival_ts'] - ticket['departure_ts']).dt.total_seconds() / 60
ticket[['departure_ts', 'arrival_ts', 'duration_min']].head()

,departure_ts,arrival_ts,duration_min
0,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,370.0
1,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1070.0
2,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,1917.0
3,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,1295.0
4,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,1306.0


Extraction de features temporelles

In [93]:
ticket['departure_hour'] = ticket['departure_ts'].dt.hour
ticket['departure_day'] = ticket['departure_ts'].dt.day_name()
ticket['departure_month'] = ticket['departure_ts'].dt.month_name()
ticket['is_weekend'] = ticket['departure_ts'].dt.dayofweek.isin([5, 6])

Délai de réservation en jours

In [94]:
ticket['booking_ts'] = (ticket['departure_ts'] - ticket['search_ts']).dt.days

ticket[['search_ts', 'departure_ts', 'booking_ts']].head()

,search_ts,departure_ts,booking_ts
0,2017-10-01 00:13:31.327000+00:00,2017-10-13 14:00:00+00:00,12
1,2017-10-01 00:13:35.773000+00:00,2017-10-13 13:05:00+00:00,12
2,2017-10-01 00:13:40.212000+00:00,2017-10-13 13:27:00+00:00,12
3,2017-10-01 00:13:40.213000+00:00,2017-10-13 13:27:00+00:00,12
4,2017-10-01 00:13:40.213000+00:00,2017-10-13 21:46:00+00:00,12


#### Valeurs abérantes

Vérification de l'existance de données abérantes.
Comparaison de la forme des tableau avant et après filtrage des données abérantes.
Suppression des valeurs abérantes si nécessaire.

Nettoyage des enregistrements avec prix négatif ou nul

In [95]:
print('Avant nettoyage :', ticket.shape)
ticket_filter = ticket[(ticket['price_in_cents'] > 0)]
print('Après nettoyage :', ticket_filter.shape)

if ticket.shape[0] == ticket_filter.shape[0]:
    print("Aucun enregistrement supprimé.")
else:
    ticket = ticket[(ticket['price_in_cents'] > 0)]

Avant nettoyage : (74168, 19)
Après nettoyage : (74168, 19)
Aucun enregistrement supprimé.


Nettyage des enregistrements avec durée négative ou nulle

In [96]:
print('Avant nettoyage :', ticket.shape)
ticket_filter = ticket[(ticket['duration_min'] > 0)]
print('Après nettoyage :', ticket_filter.shape)

if ticket.shape[0] == ticket_filter.shape[0]:
    print("Aucun enregistrement supprimé.")
else:
    ticket = ticket[(ticket['duration_min'] > 0)]

Avant nettoyage : (74168, 19)
Après nettoyage : (74168, 19)
Aucun enregistrement supprimé.


## 5. Jointures des datasets

Jointure progressive (vile -> stations -> fournisseurs)

### Jointure des données des tickets avec les villes d'origine.

In [97]:
cities_origin = cities[['id', 'local_name', 'latitude', 'longitude']].copy()
cities_origin = cities_origin.rename(columns={
    'id': 'city_id',
    'local_name': 'o_city_name',
    'latitude': 'o_lat',
    'longitude': 'o_lon'
})

"""Merge des données des tickets avec les villes d'origine."""
ticket = ticket.merge(cities_origin, left_on='o_city', right_on='city_id', how='left').drop(columns=['city_id'])


### Jointure des données des tickets avec les villes de destinations

In [98]:
cities_destination = cities[['id', 'local_name', 'latitude', 'longitude']].copy()
cities_destination = cities_destination.rename(columns={
    'id': 'city_id',
    'local_name': 'd_city_name',
    'latitude': 'd_city_latitude',
    'longitude': 'd_city_longitude'
})

"""Merge des données des tickets avec les villes de destinations."""
ticket = ticket.merge(cities_destination, left_on='d_city', right_on='city_id', how='left').drop(columns=['city_id'])

### Jointure des données avec les stations d'origine

In [99]:
stations_origin = stations[['id', 'unique_name', 'latitude', 'longitude']].copy()
stations_origin = stations_origin.rename(columns={
    'id': 'station_id',
    'unique_name': 'o_station_name',
    'latitude': 'o_station_latitude',
    'longitude': 'o_station_longitude'
})

"""Merge des données avec les stations d'origine."""
ticket = ticket.merge(stations_origin, left_on='o_station', right_on='station_id', how='left').drop(columns=['station_id'])

### Jointure des données avec les stations de destination

In [100]:
stations_destination = stations[['id', 'unique_name', 'latitude', 'longitude']].copy()
stations_destination = stations_destination.rename(columns={
    'id': 'station_id',
    'unique_name': 'd_station_name',
    'latitude': 'd_station_latitude',
    'longitude': 'd_station_longitude'
})

"""Merge des données avec les stations de destination."""
ticket = ticket.merge(stations_destination, left_on='d_station', right_on='station_id', how='left').drop(columns=['station_id'])

### Jointure des données avec les fournisseurs

In [101]:

ticket = ticket.merge(providers[['id', 'name', 'transport_type', 'has_wifi', 'has_plug', 'has_bicycle']], left_on='company', right_on='id', how='left', suffixes=('', '_provider')).drop(columns=['id_provider']).rename(columns={'name': 'provider_name'})

Colonnes finales du dataset:

In [102]:
print(ticket.columns.tolist())

print('Aperçu des données enrichies:')
ticket[['o_city_name', 'd_city_name', 'provider_name', 'transport_type', 'price_eur', 'duration_min']].head(10)

['id', 'company', 'o_station', 'd_station', 'departure_ts', 'arrival_ts', 'price_in_cents', 'search_ts', 'middle_stations', 'other_companies', 'o_city', 'd_city', 'price_eur', 'duration_min', 'departure_hour', 'departure_day', 'departure_month', 'is_weekend', 'booking_ts', 'o_city_name', 'o_lat', 'o_lon', 'd_city_name', 'd_city_latitude', 'd_city_longitude', 'o_station_name', 'o_station_latitude', 'o_station_longitude', 'd_station_name', 'd_station_latitude', 'd_station_longitude', 'provider_name', 'transport_type', 'has_wifi', 'has_plug', 'has_bicycle']
Aperçu des données enrichies:


,o_city_name,d_city_name,provider_name,transport_type,price_eur,duration_min
0,"Orléans, Centre-Val de Loire, France","Montpellier, Occitanie, France",bbc,carpooling,45.5,370.0
1,"Orléans, Centre-Val de Loire, France","Montpellier, Occitanie, France",ouibus,bus,14.5,1070.0
2,"Orléans, Centre-Val de Loire, France","Montpellier, Occitanie, France",corailintercite,train,74.0,1917.0
3,"Orléans, Centre-Val de Loire, France","Montpellier, Occitanie, France",corailintercite,train,135.0,1295.0
4,"Orléans, Centre-Val de Loire, France","Montpellier, Occitanie, France",coraillunea,train,77.1,1306.0
5,"Paris, Île-de-France, France","Lille, Hauts-de-France, France",bbc,carpooling,18.0,180.0
6,"Paris, Île-de-France, France","Lille, Hauts-de-France, France",bbc,carpooling,21.5,150.0
7,"Paris, Île-de-France, France","Lille, Hauts-de-France, France",bbc,carpooling,17.0,150.0
8,"Paris, Île-de-France, France","Lille, Hauts-de-France, France",bbc,carpooling,17.0,170.0
9,"Paris, Île-de-France, France","Lille, Hauts-de-France, France",bbc,carpooling,19.0,170.0


## 6. Calcul de la distance

### Calcul de la distance pour chaque trajet
Utilisation de la **formule de Haversine** pour calculer la distance entre deux coordonnées GPS

In [103]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calcule la distance en kilomètres entre deux point GPS
    en utilisant la formule de Haversine.
    """

    # gestion des valeurs manquantes
    if pd.isna(lon1) or pd.isna(lat1) or pd.isna(lon2) or pd.isna(lat2):
        return np.nan
    
    # conversion des degrés en radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Formule de Haversine
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6371  # Rayon de la Terre en kilomètres
    return c * km

Calcul de la distance pour chaque trajet

In [104]:
ticket['distance_km'] = ticket.apply(lambda row: haversine(row['o_lon'], row['o_lat'], row['d_city_longitude'], row['d_city_latitude']), axis=1)
print("Statistiques descriptives de la distance en km:")
print(ticket['distance_km'].describe())

Statistiques descriptives de la distance en km:
count    74168.00
mean       363.04
std        194.93
min         18.91
25%        205.84
50%        338.32
75%        480.41
max       1870.17
Name: distance_km, dtype: float64


### Création des catégories de distance

In [105]:
ticket['distance_range'] = pd.cut(ticket['distance_km'], bins=[0, 200, 800, 2000, 10000], labels=['0-200 km', '201-800 km', '801-2000 km', '2000+ km)'])
print("Répartition des tickets par tranche de distance:")
print(ticket['distance_range'].value_counts().sort_index())

Répartition des tickets par tranche de distance:
distance_range
0-200 km       13724
201-800 km     58877
801-2000 km     1567
2000+ km)          0
Name: count, dtype: int64


### Calcul de la vitesse moyenne

In [106]:
ticket['vitesse_kmh'] = (ticket['distance_km'] / (ticket['duration_min'] / 60))

print("Vitesse moyenne par type de transport:")
print(ticket.groupby('transport_type')['vitesse_kmh'].mean().sort_values(ascending=False))

Vitesse moyenne par type de transport:
transport_type
train         75.57
carpooling    75.04
bus           35.77
Name: vitesse_kmh, dtype: float64


## 7. Analyse statistiques

### 7.1 Statistiques générales

#### Prix

In [107]:
print(f"Prix minimum: {ticket['price_eur'].min():.2f} €")
print(f"Prix moyen: {ticket['price_eur'].mean():.2f} €")
print(f"Prix maximum: {ticket['price_eur'].max():.2f} €")

Prix minimum: 3.00 €
Prix moyen: 43.83 €
Prix maximum: 385.50 €


#### Durée

In [108]:
print(f"Durée minimale: {ticket['duration_min'].min():.0f} minutes ({ticket['duration_min'].min()/60:.1f}h)")
print(f"Durée moyenne: {ticket['duration_min'].mean():.0f} minutes ({ticket['duration_min'].mean()/60:.1f}h)")
print(f"Durée maximale: {ticket['duration_min'].max():.0f} minutes ({ticket['duration_min'].max()/60:.1f}h)")


Durée minimale: 20 minutes (0.3h)
Durée moyenne: 425 minutes (7.1h)
Durée maximale: 29571 minutes (492.9h)


#### Distance

In [109]:
print(f"Distance minimale: {ticket['distance_km'].min():.0f} km")
print(f"Distance moyenne: {ticket['distance_km'].mean():.0f} km")
print(f"Distance maximale: {ticket['distance_km'].max():.0f} km")


Distance minimale: 19 km
Distance moyenne: 363 km
Distance maximale: 1870 km


### 7.2 Analyse par Type de Transport

In [110]:
transport_stats = ticket.groupby('transport_type').agg({
    'price_eur': ['min', 'mean', 'max'],
    'duration_min': ['min', 'mean', 'max'],
    'distance_km': ['min', 'mean', 'max'],
    'vitesse_kmh': ['mean']
}).round(2)

transport_stats.columns = ['_'.join(col).strip() for col in transport_stats.columns.values]
print(transport_stats)

                price_eur_min  price_eur_mean  price_eur_max  \
transport_type                                                 
bus                       8.5           36.52          229.0   
carpooling                3.0           27.42          161.5   
train                     4.9           85.07          385.5   

                duration_min_min  duration_min_mean  duration_min_max  \
transport_type                                                          
bus                         65.0             938.00           29571.0   
carpooling                  20.0             246.64            1750.0   
train                       39.0             440.06            2907.0   

                distance_km_min  distance_km_mean  distance_km_max  \
transport_type                                                       
bus                       27.38            474.42          1870.17   
carpooling                18.91            306.68          1754.30   
train                     19.28  

### 7.3 Analyse par Distance et Type de Transport

Prix et durée par distance et par type de transport

In [111]:
analysis_by_distance = ticket.groupby(['distance_range', 'transport_type']).agg({
    'price_eur': ['min', 'mean', 'max'],
    'duration_min': ['min', 'mean', 'max'],
    'vitesse_kmh': ['mean']
}).round(2)
print(analysis_by_distance)

                              price_eur                duration_min           \
                                    min    mean    max          min     mean   
distance_range transport_type                                                  
0-200 km       bus                 8.50   21.83  229.0         65.0   613.95   
               carpooling          3.00   11.77  128.5         20.0   117.12   
               train               4.90   34.88  251.0         39.0   265.16   
201-800 km     bus                10.00   34.95  224.8        149.0   904.49   
               carpooling          8.50   32.18  138.0         80.0   285.79   
               train              14.00   91.25  385.5         68.0   460.66   
801-2000 km    bus                22.90   69.50  174.0        735.0  1652.17   
               carpooling         44.96   86.27  161.5        470.0   786.01   
               train              19.40  154.64  375.5        213.0   729.52   
2000+ km)      bus                  NaN 

### Tableau pivot

In [112]:
print("\n")
pivot_price = ticket.pivot_table(values='price_eur', index='distance_range', columns='transport_type', aggfunc='mean').round(2)
print("Prix moyen par type de transport et tranche de distance:")
print(pivot_price)

print("\n")
pivot_duration = ticket.pivot_table(values='duration_min', index='distance_range', columns='transport_type', aggfunc='mean').round(2)
print("Durée moyenne par type de transport et tranche de distance:")
print(pivot_duration)



Prix moyen par type de transport et tranche de distance:
transport_type    bus  carpooling   train
distance_range                           
0-200 km        21.83       11.77   34.88
201-800 km      34.95       32.18   91.25
801-2000 km     69.50       86.27  154.64


Durée moyenne par type de transport et tranche de distance:
transport_type      bus  carpooling   train
distance_range                             
0-200 km         613.95      117.12  265.16
201-800 km       904.49      285.79  460.66
801-2000 km     1652.17      786.01  729.52


## 8. Visualisations

### 8.1 Distribution des prix par type de transport

In [113]:
fig1 = px.box(ticket, x='transport_type', y='price_eur', color='transport_type' ,title='Distribution des prix par type de transport', labels={'price_eur': 'Prix (€)', 'transport_type': 'Type de transport'},
             color_discrete_map={
                 'train': 'blue',
                 'bus': 'green',
                 'carpooling': 'orange',
             })
fig1.update_layout(showlegend=False, height=500)
fig1.show()

### 8.2 Prix moyen selon la Distance et le Type de Transport

In [114]:
price_by_distance = ticket.groupby(['distance_range', 'transport_type'])['price_eur'].mean().reset_index()

fig2 = px.bar(price_by_distance, x='distance_range', y='price_eur', color='transport_type', barmode='group',
               title='Prix moyen par type de transport et tranche de distance',
               labels={'distance_range': 'Tranche de distance', 'price_eur': 'Prix moyen (€)', 'transport_type': 'Type de transport'},
               color_discrete_map={
                   'train': 'blue',
                   'bus': 'green',
                   'carpooling': 'orange',
               })
fig2.update_layout(height=500)
fig2.show()

### 8.3 Durée moyenne selon la Distance et le Type de Transport

In [115]:
duration_by_distance = ticket.groupby(['distance_range', 'transport_type'])['duration_min'].mean().reset_index()

fig3 = px.bar(duration_by_distance, x='distance_range', y='duration_min', color='transport_type', barmode='group',
               title='Durée moyenne par type de transport et tranche de distance',
               labels={'distance_range': 'Tranche de distance', 'duration_min': 'Durée moyenne (minutes)', 'transport_type': 'Type de transport'},
               color_discrete_map={
                   'train': 'blue',
                   'bus': 'green',
                   'carpooling': 'orange',
               })
fig3.update_layout(height=500)
fig3.show()

### 8.4 Relation Prix-Distance (Scatter Plot)

In [116]:
sample_data = ticket.sample(min(5000, len(ticket)), random_state=42)

fig4 = px.scatter(sample_data, x='distance_km', y='price_eur', color='transport_type',
                  title='Prix vs Distance par type de transport',
                  labels={'distance_km': 'Distance (km)', 'price_eur': 'Prix (€)', 'transport_type': 'Type de transport'},
                  color_discrete_map={
                      'train': 'blue',
                      'bus': 'green',
                      'carpooling': 'orange',
                  })
fig4.update_layout(height=500)
fig4.show()

#### 8.5 Prix selon le jour de la semaine

In [117]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ticket['departure_day_cat'] = pd.Categorical(ticket['departure_day'], categories=day_order, ordered=True)

price_by_day = ticket.groupby(['departure_day_cat'])['price_eur'].mean().reset_index()

fig5 = px.bar(
    price_by_day,
    x='departure_day_cat',
    y='price_eur',
    title='Prix moyen selon le jour de la semaine',
    labels={'departure_day_cat':'Jour', 'price_eur':'Prix Moyen (€)'},
    color='price_eur',
    color_continuous_scale='RdYlGn_r'
)

fig5.update_layout(height=400, showlegend=False)
fig5.show()

## 9. Analyses avancées

### 9.1 Modèle de prédiction du prix

In [122]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Préparation des données pour la prédiction
prediction_data = ticket[['distance_km', 'duration_min', 'departure_hour', 'booking_ts', 'price_eur']].dropna()

# Encodage du type de transport
transport_dummies = pd.get_dummies(ticket.loc[prediction_data.index, 'transport_type'], prefix='transport')
prediction_data = pd.concat([prediction_data, transport_dummies], axis=1)

# Features et target
X = prediction_data.drop('price_eur', axis=1)
y = prediction_data['price_eur']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement
model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction
y_pred = model.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score: {r2:.3f}")
print(f"Erreur moyenne absolue: {mae:.2f}€")

# Importance des features
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': abs(model.coef_)
}).sort_values('Importance', ascending=False)

print("\n Importance des variables:")
print(feature_importance.head(10))

R² Score: 0.644
Erreur moyenne absolue: 11.36€

 Importance des variables:
                Feature  Importance
6       transport_train    3.41e+01
4         transport_bus    2.00e+01
5  transport_carpooling    1.41e+01
0           distance_km    9.58e-02
2        departure_hour    8.39e-02
3            booking_ts    2.72e-02
1          duration_min    1.13e-03


##### Evaluation
64% de la variation des prix s'explique par les variables.
Si le prix réel est de 45€, le modèle prédira un prix entre 34 et 56€, soit une marge d'erreur de 11.36€ environ.

##### Importance variable (changement de prix):
En se basant sur le transport le moins chers (convoiturage) :
Transport train : 34 €, c'est-à-dire qu'un train coûte 34€ de plus
Transport Bus : 20 €, le bus coûte 20 € de plus
distance km : 0.1€, chaque KM coûte 10 centimes
departue hour: 0.08€: chaque heure de retard coûte 8 centimes

### 9.2 Carte intereactive des routes

In [133]:
# Créer un échantillon de routes
routes_map = ticket.groupby(['o_city_name', 'd_city_name', 'o_lat', 'o_lon', 'd_city_latitude', 'd_city_longitude']).agg({
    'id':'count',
    'price_eur':'mean'
}).reset_index().rename(columns={'id':'nb_tickets'})

# Prendre les 100 routes les plus populaires
routes_map = routes_map.nlargest(100, 'nb_tickets')

# Créer les points de départ
origins = routes_map[['o_city_name', 'o_lat', 'o_lon', 'nb_tickets']].copy()
origins.columns = ['city', 'lat', 'lon', 'tickets']

fig_map = px.scatter_geo(
    origins,
    lat = 'lat',
    lon = 'lon',
    size = 'tickets',
    hover_name= 'city',
    title = 'Carte des villes les plus desservies',
    scope='europe',
    size_max=30
)

fig_map.update_layout(height= 500)
fig_map.show()

## 10. Rapport des données manquantes et recommandations

#### Problèmes identifiés

##### Données de population manquantes

In [135]:
print(f" - {cities['population'].isna().sum()} villes sur {len(cities)} sans informations sur la population")
print(f" - Taux de remplissage: {(cities['population'].notna().sum() / len(cities) * 100):.1f}%")

 - 7671 villes sur 8040 sans informations sur la population
 - Taux de remplissage: 4.6%


##### Information de stations manquantes

In [136]:
print(f" - {ticket['o_station'].isna().sum()} tickets sans station d'origine")
print(" Cela concerne principalement les convoiturage.")

 - 41441 tickets sans station d'origine
 Cela concerne principalement les convoiturage.


#### RECOMMANDATIONS

Pour enrichir les données : 
Utiliser les API de INSEE pur obtenir les populations.
Utiliser OpenStreetMap pour la géolocalisation précise.